# 8.2　《分析》データを使って対策すべきネガティブなユーザー体験を把握しよう

## 8.2.2　データの理解

#### コード8.1


In [ ]:
!pip install japanize_matplotlib

#### コード8.2


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import japanize_matplotlib
import seaborn as sns
import unicodedata
import re
import requests
import collections

#### コード8.3


In [ ]:
review_df = pd.read_csv('モバイルアプリのアンケート.csv')

#### コード8.4


In [ ]:
review_df.head()

#### コード8.5

In [ ]:
review_df.shape

## 8.2.3　データの前処理


### テキストクリーニング

unicode正規化

#### コード8.6


In [ ]:
sample_text = "１２３ａｂｃｱｲｳｴｵ①②③¹²³💛"
normalized_text = unicodedata.normalize('NFKC', sample_text)
print(normalized_text)

#### コード8.7


In [ ]:
review_df['アプリレビュー_NFKC'] = review_df['アプリレビュー'].apply(lambda x: unicodedata.normalize('NFKC', x))
review_df.head()

絵文字削除

#### コード8.8


In [ ]:
!pip install demoji

#### コード8.9


In [ ]:
import demoji

sample_text = "123abcアイウエオ123123💛"
demoji.replace(string=sample_text, repl='')

#### コード8.10


In [ ]:
review_df['アプリレビュー_NFKC_demoji'] = review_df['アプリレビュー_NFKC'].apply(lambda x: demoji.replace(string=x, repl=''))
review_df.head()

改行を削除

#### コード8.11


In [ ]:
sample_text = "123abcアイウエオ\n123123"
sample_text.replace('\n','')

#### コード8.12


In [ ]:
review_df['アプリレビュー_NFKC_demoji_nr'] = review_df['アプリレビュー_NFKC_demoji'].apply(lambda x: x.replace('\n',''))
review_df.head()

特殊文字削除

#### コード8.13


In [ ]:
review_df['アプリレビュー_NFKC_demoji_nr_spemoji'] = review_df['アプリレビュー_NFKC_demoji_nr'].apply(lambda x: re.sub('[\uFF01-\uFF0F\uFF1A-\uFF20\uFF3B-\uFF40\uFF5B-\uFF65\u3000-\u303F]', '', x))
review_df.head()

大文字を小文字に統一

#### コード8.14


In [ ]:
sample_text = "！／：123abc〒アイ〻ウエオ123123＠"
re.sub('[\uFF01-\uFF0F\uFF1A-\uFF20\uFF3B-\uFF40\uFF5B-\uFF65\u3000-\u303F]', '', sample_text)

#### コード8.15


In [ ]:
sample_text = "aBcDEFgHiJk"
sample_text.lower()

#### コード8.16


In [ ]:
review_df['アプリレビュー_NFKC_demoji_nr_spemoji_lower'] = review_df['アプリレビュー_NFKC_demoji_nr_spemoji'].apply(lambda x: x.lower())
review_df.head()

#### 形態素解析

#### コード8.17


In [ ]:
! pip install mecab-python3 unidic-lite

#### コード8.18


In [ ]:
import MeCab
m = MeCab.Tagger()
sample_text = "MeCabを使った形態素解析のサンプルコードを作成しています。"
parsed_text = m.parse(sample_text)
print(parsed_text)

#### コード8.19


In [ ]:
word_sequence = str()

m.parse('')
node = m.parseToNode(sample_text)

while node is not None:
    word = node.surface
    pos = node.feature.split(",")[1]
    if pos in ['普通名詞']:
        word_sequence = word_sequence + word + " "
    node = node.next
print(word_sequence)

#### コード8.20


In [ ]:
stopwords = ['サンプル']
word_list = []
for word in word_sequence.split(' '):
    if word not in stopwords:
        word_list.append(word)
word_list

#### コード8.21


In [ ]:
def mecab_analysis(text):
    word_sequence = str()
    try:
        m.parse('')
        node = m.parseToNode(text)

        while node:
            word = node.surface
            pos = node.feature.split(",")[1]
            if pos not in ['格助詞', '接続助詞', '句点', '読点']:
                word_sequence = word_sequence + word + " "
            node = node.next
        return word_sequence
    except:
        return word_sequence

#### コード8.22


In [ ]:
#10 / 0

#### コード8.23


In [ ]:
try:
    print(10 / 0)
except:
    print("エラーが発生しました")

#### コード8.24


In [ ]:
review_df['アプリレビュー_NFKC_tokens'] = review_df['アプリレビュー_NFKC_demoji_nr_spemoji_lower'].apply(mecab_analysis)
review_df[['アプリレビュー_NFKC_demoji_nr_spemoji_lower', 'アプリレビュー_NFKC_tokens']].head()

#### ストップワードの除去

#### コード8.25


In [ ]:
stopwords =  ['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ', 'これ', 'これら', 'ごろ', 'さまざま', 'さらい', 'さん', 'しかた', 'しよう', 'すか', 'ずつ', 'すね', 'すべて', 'ぜんぶ', 'そう', 'そこ', 'そちら', 'そっち', 'そで', 'それ', 'それぞれ', 'それなり', 'たくさん', 'たち', 'たび', 'ため', 'だめ', 'ちゃ', 'ちゃん', 'てん', 'とおり', 'とき', 'どこ', 'どこか', 'ところ', 'どちら', 'どっか', 'どっち', 'どれ', 'なか', 'なかば', 'なに', 'など', 'なん', 'はじめ', 'はず', 'はるか', 'ひと', 'ひとつ', 'ふく', 'ぶり', 'べつ', 'へん', 'ぺん', 'ほう', 'ほか', 'まさ', 'まし', 'まとも', 'まま', 'みたい', 'みつ', 'みなさん', 'みんな', 'もと', 'もの', 'もん', 'やつ', 'よう', 'よそ', 'わけ', 'わたし', 'ハイ', 'です', 'ます', ' ', '', 'に', 'で', 'ご', 'つい', 'でし', 'た', 'は', 'も', 'の', 'ん', 'か', 'れ', 'お', 'この', 'な', 'し', 'ませ', 'でき', 'あり','する', 'い', 'なり', 'さ', 'ね']

#### コード8.26


In [ ]:
def remove_stopwords(text):
    word_list = []
    for word in text.split(' '):
        if word not in stopwords:
            word_list.append(word)
    return word_list

#### コード8.27


In [ ]:
review_df['アプリレビュー_NFKC_tokens_stopwords'] = review_df['アプリレビュー_NFKC_tokens'].apply(remove_stopwords)
review_df['アプリレビュー_NFKC_tokens_stopwords'].head()

### 8.2.4　数値化（集計）

#### コード8.28


In [ ]:
words = review_df['アプリレビュー_NFKC_tokens_stopwords'].values.tolist()

words_list = [val for sublist in words for val in sublist]
words_list[:10]

#### コード8.29


In [ ]:
c = collections.Counter(words_list)
c.most_common()[:10]

#### コード8.30


In [ ]:
fig = plt.subplots(figsize=(8, 10))
sns.countplot(y=words_list,order=[i[0] for i in c.most_common(10)],palette="Blues_r")

#### コード8.31


In [ ]:
text_row = review_df[review_df['アプリレビュー_NFKC_tokens_stopwords'].apply(lambda x: True if 'ない' in x else False)]['アプリレビュー']

for text in text_row:
    print(text)

バイグラム

#### コード8.32


In [ ]:
def bi_gram(target):
    n = 2
    word_list = []
    # 基準を1文字(単語)ずつ ずらしながらn文字分抜き出す
    for idx in range(len(target) - n + 1):
        word_list.append('-'.join(target[idx:idx + n]))
    return word_list

#### コード8.33


In [ ]:
review_df['アプリレビュー_NFKC_tokens_stopwords_bigram'] = review_df['アプリレビュー_NFKC_tokens_stopwords'].apply(bi_gram)

#### コード8.34


In [ ]:
bi_words = review_df['アプリレビュー_NFKC_tokens_stopwords_bigram'].values.tolist()

bi_words_list = [val for sublist in bi_words for val in sublist]
bi_words_list[:10]

#### コード8.35


In [ ]:
c = collections.Counter(bi_words_list)
fig = plt.subplots(figsize=(8, 10))

sns.countplot(y=bi_words_list,order=[i[0] for i in c.most_common(10)],palette="Blues_r")

#### コード8.36


In [ ]:
#「強制終了」が多いので、具体的にどのような「強制終了」が問題なのか確認
text_row = review_df[review_df['アプリレビュー_NFKC_tokens_stopwords_bigram'].apply(lambda x: True if '強制-終了' in x else False)]['アプリレビュー']

for text in text_row:
    print(text)

#### コード8.37


In [ ]:
#「友達-コード」が多いので、具体的にどのような「友達-コード」が問題なのか確認
text_row = review_df[review_df['アプリレビュー_NFKC_tokens_stopwords_bigram'].apply(lambda x: True if '友達-コード' in x else False)]['アプリレビュー']

for text in text_row:
    print(text)